In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
import time

In [2]:
# Домашнее задание к лекции "Основы веб-скрапинга"
# Обязательная часть
# Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска 
# (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

# <дата> - <заголовок> - <ссылка на материал>
# В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, 
# если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)
# Функция из обязательной части задания должна быть расширена следующим образом:

# кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить 
# материалы с первых 4 страниц результатов;
# в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
# <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [3]:
# Решение обязательная часть
def habr_search(query):
    output = pd.DataFrame() # результаты
    for request in query:
        url = 'https://habr.com/ru/search/' # ссылка на habr
        params = {'q': request} # параметры
        res = requests.get(url, params=params) # затягиваем данные
        soup = BeautifulSoup(res.text) # варим суп
        articles = soup.find_all('article', class_='tm-articles-list__item') # все статьи
        
        for article in articles:
            date = datetime.strftime(datetime.strptime(article.find('span', class_='tm-article-snippet__datetime-published').time.get('datetime'), 
            '%Y-%m-%dT%H:%M:%S.000Z'), '%Y-%m-%d') # дата
            title = article.find('a', class_='tm-article-snippet__title-link').text # тема
            short_link = article.find('a', class_='tm-article-snippet__title-link').get('href') # краткая ссылка
            link = 'https://habr.com'+short_link # ссылка
            line = {'date': date, 'title': title, 'link': link}
            output = pd.concat([output, pd.DataFrame([line])])
            output.drop_duplicates('link', keep='first', inplace=True) # убрать дубликаты
            # output['dublicated'] = output.duplicated() # проверка на дубликаты
    return output.reset_index(drop=True) 

habr_search(['linux', 'python'])

,date,title,link
0,2020-05-06,Для ядра Linux предложили систему прозрачного ...,https://habr.com/ru/company/itsumma/news/t/500...
1,2021-04-19,В ядро Linux добавили начальную поддержку проц...,https://habr.com/ru/company/dcmiran/news/t/553...
2,2022-11-24,«Администрирование Linux. Мега» — старт третье...,https://habr.com/ru/company/southbridge/news/t...
3,2020-08-19,Kali Linux получил графический интерфейс для п...,https://habr.com/ru/news/t/515726/
4,2022-10-28,«Администрирование Linux. Мега» — стартуем уже...,https://habr.com/ru/company/southbridge/news/t...
5,2022-11-01,Минцифры планирует определить три наиболее пер...,https://habr.com/ru/news/t/696678/
6,2020-06-28,Вышел Linux Mint 20 «Ulyana» (Ульяна),https://habr.com/ru/news/t/508570/
7,2021-01-19,Corellium выпустила прототип порта Linux для с...,https://habr.com/ru/news/t/538148/
8,2021-03-27,"Линус Торвальдс рассказал о том, где Rust впиш...",https://habr.com/ru/news/t/549266/
9,2021-08-13,"Steam Deck будет работать на Arch Linux, а не ...",https://habr.com/ru/company/ruvds/news/t/572752/


In [4]:
# Решение дополнительная часть (частично)
# Как достать основной текст так и не разобрался
# там где он вроде должен быть не достаётся ничего:
#   soup.find_all('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1')
# буду благодарен за комментарии, как его нужно было доставать

def habr_search(query, page=1):
    output = pd.DataFrame() # результаты
    for request in query:
        params = {'q': request} # параметры
        
        for i in range(1, page+1):
            url = 'https://habr.com/ru/search/'+'page'+str(i)+'/' # ссылка на habr
            res = requests.get(url, params) # затягиваем данные
            soup = BeautifulSoup(res.text) # варим суп
            articles = soup.find_all('article', class_='tm-articles-list__item') # все статьи
            time.sleep(0.33) # торопиться не надо
            for article in articles:
                date = datetime.strftime(datetime.strptime(article.find('span', class_='tm-article-snippet__datetime-published').time.get('datetime'), 
                '%Y-%m-%dT%H:%M:%S.000Z'), '%Y-%m-%d') # дата
                title = article.find('a', class_='tm-article-snippet__title-link').text # тема
                short_link = article.find('a', class_='tm-article-snippet__title-link').get('href') # краткая ссылка
                link = 'https://habr.com'+short_link # ссылка
                line = {'date': date, 'title': title, 'link': link}
                output = pd.concat([output, pd.DataFrame([line])])
                output.drop_duplicates('link', keep='first', inplace=True) # убрать дубликаты
                # output['dublicated'] = output.duplicated() # проверка на дубликаты
    return output.reset_index(drop=True) 
    # return print(url)

habr_search(['linux', 'python'], 2)

,date,title,link
0,2020-05-06,Для ядра Linux предложили систему прозрачного ...,https://habr.com/ru/company/itsumma/news/t/500...
1,2021-04-19,В ядро Linux добавили начальную поддержку проц...,https://habr.com/ru/company/dcmiran/news/t/553...
2,2022-11-24,«Администрирование Linux. Мега» — старт третье...,https://habr.com/ru/company/southbridge/news/t...
3,2020-08-19,Kali Linux получил графический интерфейс для п...,https://habr.com/ru/news/t/515726/
4,2022-10-28,«Администрирование Linux. Мега» — стартуем уже...,https://habr.com/ru/company/southbridge/news/t...
...,...,...,...
75,2022-09-19,Асинхронное программирование на Python для джу...,https://habr.com/ru/company/kts/news/t/688988/
76,2022-11-08,Показываем полезное с Python meetup от Evrone,https://habr.com/ru/company/evrone/news/t/698054/
77,2021-12-08,"Открытый семинар: FastAPI, или как быстро доба...",https://habr.com/ru/news/t/594325/
78,2022-04-18,Онлайн-митап от руководителя практики Python U...,https://habr.com/ru/company/usetech/news/t/661...


In [5]:
# тренировка
# url = 'https://habr.com/ru/search/?q=linux&target_type=posts&order=relevance' # ссылка на habr
# res = requests.get(url) # затягиваем данные
# soup = BeautifulSoup(res.text) # варим суп
# # search = soup.find_all('div' ,class_='tm-sub-page') # вся страница со статьями
# # search = soup.find_all('div', class_='tm-articles-list') # список статей
# # search = soup.find_all('article', class_='tm-articles-list__item')
# # search = soup.find('a', class_='tm-article-snippet__title-link').text # тема
# # search = soup.find('a', class_='tm-article-snippet__title-link').get('href') # краткая ссылка
# # search = datetime.strftime(datetime.strptime(soup.find('span', class_='tm-article-snippet__datetime-published').time.get('datetime'), '%Y-%m-%dT%H:%M:%S.000Z'), '%Y-%m-%d') # дата
# search = soup.find_all('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1') # текст
# print(search)
# print(soup)

In [6]:
# datetime.strftime(str(datetime.strptime(search, '%Y-%m-%dT%H:%M:%S.000Z')), '%Y-%m-%d')